In [1]:
# Import Libraries and Dependencies
import pandas as pd

### 1. Combine and Clean the Data
#### Import CSVs

In [3]:
# Read the CSV files into DataFrames.
df_2020 = pd.read_csv('../athletic_sales_2020.csv')
df_2021 = pd.read_csv('../athletic_sales_2021.csv')

In [4]:
# Display the 2020 sales DataFrame
df_2020.head()


,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000.00,In-store
1,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68,83,5644,2426.92,Online
2,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,75,275,206250,61875.00,Outlet
3,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,34,384,13056,6789.12,Outlet
4,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,53,83,4399,1407.68,Outlet


In [5]:
# Display the 2021 sales DataFrame
df_2021.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,65,750,487500,121875.00,Outlet
1,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,51,233,11883,3208.41,Outlet
2,Kohl's,1189833,1/1/21,Midwest,Montana,Billings,Men's Apparel,50,275,137500,82500.00,Outlet
3,Kohl's,1189833,1/1/21,Midwest,Montana,Billings,Men's Apparel,47,77,3619,2714.25,Online
4,West Gear,1128299,1/1/21,West,California,San Francisco,Men's Athletic Footwear,64,225,14400,5184.00,Online


#### Check the data types of each DataFrame

In [9]:
# Check the 2020 sales data types.
column_names_2020 = list(df_2020.columns)
print("Column names for df_2020:", column_names_2020)

Column names for df_2020: ['retailer', 'retailer_id', 'invoice_date', 'region', 'state', 'city', 'product', 'price_per_unit', 'units_sold', 'total_sales', 'operating_profit', 'sales_method']


In [10]:
# Check the 2021 sales data types.
column_names_2021 = list(df_2021.columns)
print("Column names for df_2021:", column_names_2021)

Column names for df_2021: ['retailer', 'retailer_id', 'invoice_date', 'region', 'state', 'city', 'product', 'price_per_unit', 'units_sold', 'total_sales', 'operating_profit', 'sales_method']


#### Combine the sales data by rows.

In [11]:
# Combine the 2020 and 2021 sales DataFrames on the rows and reset the index.
# Combine the 2020 and 2021 DataFrames on the rows
combined_df = pd.concat([df_2020, df_2021], ignore_index=True)

# Reset the index, although `ignore_index=True` in concat has already done this effectively
combined_df.reset_index(drop=True, inplace=True)

# Display the combined DataFrame
combined_df.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
0,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,50,1200,600000,300000.00,In-store
1,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,68,83,5644,2426.92,Online
2,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,75,275,206250,61875.00,Outlet
3,Foot Locker,1185732,1/1/20,Northeast,New York,New York,Men's Street Footwear,34,384,13056,6789.12,Outlet
4,Foot Locker,1185732,1/1/20,Northeast,Pennsylvania,Philadelphia,Women's Apparel,53,83,4399,1407.68,Outlet


In [15]:
# Check if any values are null.
has_nulls_anywhere = combined_df.isnull().values.any()
print(has_nulls_anywhere)

False


In [18]:
# Check the data type of each column
column_names = combined_df.columns
print(column_names)

Index(['retailer', 'retailer_id', 'invoice_date', 'region', 'state', 'city',
       'product', 'price_per_unit', 'units_sold', 'total_sales',
       'operating_profit', 'sales_method'],
      dtype='object')


In [20]:
# Convert the "invoice_date" to a datetime datatype
combined_df['invoice_date'] = pd.to_datetime(combined_df['invoice_date'], format='%Y-%m-%d')

In [25]:
# Confirm that the "invoice_date" data type has been changed.
print(combined_df['invoice_date'].dtype)

datetime64[ns]


### 2. Determine which Region Sold the Most Products

#### Using `groupby`

In [27]:
# Show the number products sold for region, state, and city.
# Rename the sum to "Total_Products_Sold".

product_sales = combined_df.groupby(['region', 'state', 'city'])['units_sold'].sum().reset_index(name='Total_Products_Sold')

# Sort the results by 'Total_Products_Sold' in descending order
top_product_sales = product_sales.sort_values(by='Total_Products_Sold', ascending=False)

# Show the top 5 results
top_5_regions = top_product_sales.head(5)
print(top_5_regions)



                            

       region       state           city  Total_Products_Sold
21  Northeast    New York       New York               111954
33      South       Texas        Houston                90322
44       West  California  San Francisco                85478
43       West  California    Los Angeles                76384
34  Southeast     Florida          Miami                73135


#### Using `pivot_table`

In [28]:
# Show the number products sold for region, state, and city.

# Create a pivot table to show the sum of products sold for each region, state, and city
product_sales_pivot = combined_df.pivot_table(values='units_sold',
                                              index=['region', 'state', 'city'],
                                              aggfunc='sum').rename(columns={'units_sold': 'Total_Products_Sold'})

# Sort the results by 'Total_Products_Sold' in descending order
top_product_sales_pivot = product_sales_pivot.sort_values(by='Total_Products_Sold', ascending=False)

# Show the top 5 results
top_5_regions_pivot = top_product_sales_pivot.head(5)
print(top_5_regions_pivot)






                                    Total_Products_Sold
region    state      city                              
Northeast New York   New York                    111954
South     Texas      Houston                      90322
West      California San Francisco                85478
                     Los Angeles                  76384
Southeast Florida    Miami                        73135


### 3. Determine which Region had the Most Sales

#### Using `groupby`

In [31]:

# Group by 'region', 'state', and 'city', then sum the 'total_sales' column to get total sales
region_sales_groupby = combined_df.groupby(['region', 'state', 'city'])['total_sales'].sum()

# Rename the Series to 'Total Sales'
region_sales_groupby.name = 'Total Sales'

# Sort the Series by 'Total Sales' in descending order
top_region_sales_groupby = region_sales_groupby.sort_values(ascending=False)

# Show the top 5 results
top_5_sales_regions_groupby = top_region_sales_groupby.head(5)
print(top_5_sales_regions_groupby)


region     state           city         
Northeast  New York        New York         39801235
West       California      San Francisco    33973228
Southeast  Florida         Miami            31600863
           South Carolina  Charleston       29285637
           Florida         Orlando          27682851
Name: Total Sales, dtype: int64


#### Using `pivot_table`

In [30]:
# Show the total sales for the products sold for each region, state, and city.

region_sales_pivot = combined_df.pivot_table(values='total_sales', 
                                             index=['region', 'state', 'city'], 
                                             aggfunc='sum')

# Optionally rename the 'total_sales' column to 'Total Sales'
region_sales_pivot.columns = ['Total Sales']

# Sort the results by 'Total Sales' in descending order
top_region_sales_pivot = region_sales_pivot.sort_values(by='Total Sales', ascending=False)

# Display the top 5 regions with the highest total sales
top_5_sales_regions_pivot = top_region_sales_pivot.head(5)
print(top_5_sales_regions_pivot)


                                        Total Sales
region    state          city                      
Northeast New York       New York          39801235
West      California     San Francisco     33973228
Southeast Florida        Miami             31600863
          South Carolina Charleston        29285637
          Florida        Orlando           27682851


### 4. Determine which Retailer had the Most Sales

#### Using `groupby`

In [32]:
# Show the total sales for the products sold for each retailer, region, state, and city.
# Using groupby to sum up the total sales for each combination of retailer, region, state, and city
retailer_sales_groupby = combined_df.groupby(['retailer', 'region', 'state', 'city'])['total_sales'].sum()

# Rename the "total_sales" column to "Total Sales"
retailer_sales_groupby.name = 'Total Sales'

# Sort the results by 'Total Sales' in descending order to identify the top retailers
top_retailer_sales_groupby = retailer_sales_groupby.sort_values(ascending=False)

# Show the top 5 results
top_5_retailer_sales = top_retailer_sales_groupby.head(5)
print(top_5_retailer_sales)


retailer     region     state           city         
West Gear    West       California      San Francisco    32794405
Kohl's       West       California      Los Angeles      25127160
Foot Locker  Northeast  New York        New York         25008568
West Gear    West       Washington      Seattle          24862675
Foot Locker  Southeast  South Carolina  Charleston       24822280
Name: Total Sales, dtype: int64


#### Using `pivot_table`

In [33]:
# Show the total sales for the products sold for each retailer, region, state, and city.
# Create a pivot table to sum up the total sales for each combination of retailer, region, state, and city
retailer_sales_pivot = combined_df.pivot_table(values='total_sales',
                                               index=['retailer', 'region', 'state', 'city'],
                                               aggfunc='sum')

# Rename the "total_sales" column to "Total Sales"
retailer_sales_pivot.columns = ['Total Sales']

# Sort the results by 'Total Sales' in descending order to identify the top retailers
top_retailer_sales_pivot = retailer_sales_pivot.sort_values(by='Total Sales', ascending=False)

# Show the top 5 results
top_5_retailer_sales_pivot = top_retailer_sales_pivot.head(5)
print(top_5_retailer_sales_pivot)


                                                    Total Sales
retailer    region    state          city                      
West Gear   West      California     San Francisco     32794405
Kohl's      West      California     Los Angeles       25127160
Foot Locker Northeast New York       New York          25008568
West Gear   West      Washington     Seattle           24862675
Foot Locker Southeast South Carolina Charleston        24822280


### 5. Determine which Retailer Sold the Most Women's Athletic Footwear

In [38]:
# Filter the sales data to get the women's athletic footwear sales data.

womens_athletic_footwear_df = combined_df.loc[combined_df['product'] == "Women's Athletic Footwear"]
womens_athletic_footwear_df.head()

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,sales_method
21,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,36,281,10116,3742.92,Outlet
22,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,41,247,10127,4658.42,Online
23,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45,850,382500,133875.00,In-store
26,West Gear,1128299,2020-01-05,Northeast,Pennsylvania,Philadelphia,Women's Athletic Footwear,45,300,135000,47250.00,Outlet
27,West Gear,1128299,2020-01-05,Northeast,Pennsylvania,Philadelphia,Women's Athletic Footwear,34,90,3060,1254.60,Outlet


#### Using `groupby`

In [40]:
# Group by 'retailer', 'region', 'state', and 'city', then sum the 'units_sold' for women's athletic footwear
retailer_footwear_sales = womens_athletic_footwear_df.groupby(['retailer', 'region', 'state', 'city'])['units_sold'].sum().reset_index()

# Rename the 'units_sold' column to 'Womens_Footwear_Units_Sold'
retailer_footwear_sales.rename(columns={'units_sold': 'Womens_Footwear_Units_Sold'}, inplace=True)

# Sort the results by 'Womens_Footwear_Units_Sold' in descending order
top_retailer_footwear_sales = retailer_footwear_sales.sort_values(by='Womens_Footwear_Units_Sold', ascending=False)

# Show the top 5 results
top_5_retailer_footwear_sales = top_retailer_footwear_sales.head(5)
print(top_5_retailer_footwear_sales)

          retailer     region           state           city  \
101      West Gear       West      California  San Francisco   
22     Foot Locker  Northeast        New York       New York   
49          Kohl's       West      California    Los Angeles   
33     Foot Locker  Southeast  South Carolina     Charleston   
68   Sports Direct      South           Texas         Dallas   

     Womens_Footwear_Units_Sold  
101                       12107  
22                        10996  
49                        10826  
33                         8814  
68                         8790  


#### Using `pivot_table`

In [20]:
# Show the total number of women's athletic footwear sold for each retailer, region, state, and city.


# Rename the "units_sold" column to "Womens_Footwear_Units_Sold"

# Show the top 5 results.


### 5. Determine the Day with the Most Women's Athletic Footwear Sales

In [41]:

# Create a pivot table with 'invoice_date' as the index and 'total_sales' as the values parameter
daily_sales_pivot = womens_athletic_footwear_df.pivot_table(values='total_sales',
                                                            index='invoice_date',
                                                            aggfunc='sum')

# Rename the aggregated column to 'Daily Total Sales'
daily_sales_pivot.columns = ['Daily Total Sales']

# Resample the pivot table to daily bins and sum the total sales for each day
daily_sales_resampled = daily_sales_pivot.resample('D').sum()

# Sort the resampled DataFrame in descending order to show the top 10 days
top_10_days = daily_sales_resampled.sort_values(by='Daily Total Sales', ascending=False).head(10)

print(top_10_days)

              Daily Total Sales
invoice_date                   
2021-07-16              1521825
2021-12-16              1473497
2021-06-17              1376988
2021-08-17              1086294
2021-07-23              1021806
2021-11-17              1021145
2021-12-09               915011
2021-06-24               884238
2021-07-09               869054
2021-08-10               839120


In [45]:
# Resample the pivot table into daily bins, and get the total sales for each day.
daily_sales_resampled = daily_sales_pivot.resample('D').sum()

# Sort the resampled pivot table in ascending order on "Total Sales".
sorted_daily_sales = daily_sales_resampled.sort_values(by='Daily Total Sales', ascending=True)

print(sorted_daily_sales)

              Daily Total Sales
invoice_date                   
2021-01-01                    0
2020-10-10                    0
2020-10-09                    0
2020-10-07                    0
2020-10-06                    0
...                         ...
2021-07-23              1021806
2021-08-17              1086294
2021-06-17              1376988
2021-12-16              1473497
2021-07-16              1521825

[727 rows x 1 columns]


### 6.  Determine the Week with the Most Women's Athletic Footwear Sales

In [49]:
# Resample the daily sales data into weekly bins and sum the total sales for each week
weekly_sales_resampled = daily_sales_pivot.resample('W').sum()

# Rename the column in the resampled DataFrame
weekly_sales_resampled.rename(columns={weekly_sales_resampled.columns[0]: 'Total Sales'}, inplace=True)

# Sort the resampled pivot table in ascending order by 'Total Sales'
sorted_weekly_sales = weekly_sales_resampled.sort_values(by='Total Sales', ascending=False)

sorted_weekly_sales.head(10)

,Total Sales
invoice_date,
2021-12-19,3098970
2021-12-12,2922161
2021-07-11,2835078
2021-07-18,2801449
2021-11-14,2531721
2021-08-22,2491259
2021-08-15,2463941
2021-11-21,2449537
2021-05-16,2422132
